# FEATURE EXTRACTION

## Library Imports

In [1]:
import numpy as np
import pandas as pd
import math
import os

import scipy.stats as st
import scipy.signal as sgl
pd.set_option('display.max_columns', 500)

## Variables related to files

In [2]:
file_name=''
file_size=0

## Variables related to attributes

In [3]:
ID=[]
X=[]
Y=[]
TS=[]
BS=[]
AZ=[]
AL=[]
P=[]
V=[]
SDX=[]
SDY=[]
A=[]
SDV=[]
SDA=[]
F=[]

## Variables related to features

In [4]:
avgX=[]
avgY=[]
avgSDX=[]
avgSDY=[]
avgV=[]
avgA=[]
avgSDV=[]
avgSDA=[]
pen_down=[]
pen_up=[]
pen_ratio=[]
sign_width=[]
sign_height=[]
width_height_ratio=[]
total_sign_duration=[]
range_pressure=[]

max_pressure = []
sample_points = []
sample_points_to_width = []
mean_pressure = []
pressure_variance = []
avg_x_velocity = []
avg_y_velocity = []
max_x_velocity = []
max_y_velocity = []
samples_positive_x_velocity = []
samples_positive_y_velocity = []
variance_x_velocity = []
variance_y_velocity = []
std_x_velocity = []
std_y_velocity = []
median_x_velocity = []
median_y_velocity = []
mode_x_velocity = []
mode_y_velocity = []
corr_x_y_velocity = []
mean_x_acceleration = []
mean_y_acceleration = []
corr_x_y_acceleration = []
variance_x_acceleration = []
variance_y_acceleration = []
std_x_acceleration = []
std_y_acceleration = []
x_local_minima = []
y_local_minima = []

In [5]:
%%time
for i in range(1,41):
    for j in range(1,41):
        file_name='../signature_data_preprocessed/U'+str(i)+'S'+str(j)+'.txt'
        file=pd.read_csv(file_name,delimiter=' ',names=['X','Y','TS','BS','AZ','AL','P','V','SDX','SDY','A','SDV','SDA'],
        header=None)
        file_size=len(file)
        X=file['X']
        Y=file['Y']
        TS=file['TS']
        BS=file['BS']
        AZ=file['AZ']
        AL=file['AL']
        P=file['P']
        V=file['V']
        SDX=file['SDX']
        SDY=file['SDY']
        A=file['A']
        SDV=file['SDV']
        SDA=file['SDA']
        ID.append(i)
        avgX.append(sum(X)/file_size)
        avgY.append(sum(Y)/file_size)
        avgSDX.append(sum(SDX)/file_size)
        avgSDY.append(sum(SDY)/file_size)
        avgV.append(sum(V)/file_size)
        avgA.append(sum(A)/file_size)
        avgSDV.append(sum(SDV)/file_size)
        avgSDA.append(sum(SDA)/file_size)
        pen_down.append(sum(BS))
        pen_up.append(file_size-sum(BS))
        pen_ratio.append((sum(BS))/(file_size-sum(BS)))
        sign_width.append(max(X)-min(X))
        sign_height.append(max(Y)-min(Y))
        width_height_ratio.append((max(X)-min(X))/(max(Y)-min(Y)))
        total_sign_duration.append(TS[file_size-1]-TS[0])
        range_pressure.append(max(P)-min(P))
#         16 features till here
#         my features ()
        sample_points.append(file_size)
        sample_points_to_width.append(file_size/(max(X)-min(X)))
        max_pressure.append(max(P))
        mean_pressure.append(np.mean(P))
        pressure_variance.append(np.var(P))
        
        # calculating x, y velocities
        x_displacement = []
        y_displacement = []
        x_velocity = []
        y_velocity = []
        x_acceleration = []
        y_acceleration = []
        for k in range(0, file_size-1):
            x_displacement = X[k+1] - X[k]
            y_displacement = Y[k+1] - Y[k]
            time = TS[k+1] - TS[k]

            x_velocity.append(x_displacement/time)
            y_velocity.append(y_displacement/time)
            
            x_acceleration.append(x_displacement/(time**2))
            y_acceleration.append(y_displacement/(time**2))
        
        avg_x_velocity.append(np.mean(x_velocity))
        avg_y_velocity.append(np.mean(y_velocity))
        
        max_x_velocity.append(max(x_velocity))
        max_y_velocity.append(max(y_velocity))
        
        samples_positive_x_velocity.append(len([x for x in x_velocity if x > 0]))
        samples_positive_y_velocity.append(len([y for y in y_velocity if y > 0]))        
        
        variance_x_velocity.append(np.var(x_velocity))
        variance_y_velocity.append(np.var(y_velocity))
        
        std_x_velocity.append(np.std(x_velocity))
        std_y_velocity.append(np.std(y_velocity))
        
        median_x_velocity.append(np.median(x_velocity))
        median_y_velocity.append(np.median(y_velocity))
        
#         mode_x_velocity.append(max(set(x_velocity), key=x_velocity.count))
#         mode_y_velocity.append(max(set(y_velocity), key=y_velocity.count))
        
        corr_velocity, _ = st.pearsonr(x_velocity, y_velocity)
        corr_x_y_velocity.append(corr_velocity)
        
        mean_x_acceleration.append(np.mean(x_acceleration))
        mean_y_acceleration.append(np.mean(y_acceleration))
        
        corr_acceleration, _ = st.pearsonr(x_acceleration, y_acceleration)
        corr_x_y_acceleration.append(corr_acceleration)
        
        variance_x_acceleration.append(np.var(x_acceleration))
        variance_y_acceleration.append(np.var(y_acceleration))
        
        std_x_acceleration.append(np.std(x_acceleration))
        std_y_acceleration.append(np.std(y_acceleration))
        
        x_local_minima.append(len(sgl.argrelextrema(np.array(X), np.less)[0]))
        y_local_minima.append(len(sgl.argrelextrema(np.array(Y), np.less)[0]))

Wall time: 32.4 s


## Creating forgery attribute

In [6]:
i=1
while(i<1601):
    for j in range(1,21):
        F.append(0)
    for j in range(1,21):
        F.append(1)
    i+=40
print(len(F))

1600


## Creation of DataFrame

In [7]:
data={
    'ID':ID,
    
    'avgX':avgX,
    'avgY':avgY,
    'avgSDX':avgSDX,
    'avgSDY':avgSDY,
    'avgV':avgV,
    'avgA':avgA,
    'avgSDV':avgSDV,
    'avgSDA':avgSDA,
    'pen_down':pen_down,
    'pen_up':pen_up,
    'pen_ratio':pen_ratio,
    'sign_width':sign_width,
    'sign_height':sign_height,
    'width_height_ratio':width_height_ratio,
    'total_sign_duration':total_sign_duration,
    'range_pressure':range_pressure,
    
    'max_pressure': max_pressure,
    'sample_points': sample_points,
    'sample_points_to_width': sample_points_to_width,
    'mean_pressure': mean_pressure,
    'pressure_variance': pressure_variance,
    'avg_x_velocity': avg_x_velocity,
    'avg_y_velocity': avg_y_velocity,
    'max_x_velocity': max_x_velocity,
    'max_y_velocity': max_y_velocity,
    'samples_positive_x_velocity': samples_positive_x_velocity,
    'samples_positive_y_velocity': samples_positive_y_velocity,
    'variance_x_velocity': variance_x_velocity,
    'variance_y_velocity': variance_y_velocity,
    'std_x_velocity': std_x_velocity,
    'std_y_velocity': std_y_velocity,
    'median_x_velocity': median_x_velocity,
    'median_y_velocity': median_y_velocity,
#     'mode_x_velocity': mode_x_velocity,
#     'mode_y_velocity': mode_y_velocity,
    'corr_x_y_velocity': corr_x_y_velocity,
    'mean_x_acceleration': mean_x_acceleration,
    'mean_y_acceleration': mean_y_acceleration,
    'corr_x_y_acceleration': corr_x_y_acceleration,
    'variance_x_acceleration': variance_x_acceleration,
    'variance_y_acceleration': variance_y_acceleration,
    'std_x_acceleration': std_x_acceleration,
    'std_y_acceleration': std_y_acceleration,
    'x_local_minima': x_local_minima,
    'y_local_minima': y_local_minima,
    
    'F':F
}

In [8]:
df=pd.DataFrame(data)
df.head()

,ID,avgX,avgY,avgSDX,avgSDY,avgV,avgA,avgSDV,avgSDA,pen_down,pen_up,pen_ratio,sign_width,sign_height,width_height_ratio,total_sign_duration,range_pressure,max_pressure,sample_points,sample_points_to_width,mean_pressure,pressure_variance,avg_x_velocity,avg_y_velocity,max_x_velocity,max_y_velocity,samples_positive_x_velocity,samples_positive_y_velocity,variance_x_velocity,variance_y_velocity,std_x_velocity,std_y_velocity,median_x_velocity,median_y_velocity,corr_x_y_velocity,mean_x_acceleration,mean_y_acceleration,corr_x_y_acceleration,variance_x_acceleration,variance_y_acceleration,std_x_acceleration,std_y_acceleration,x_local_minima,y_local_minima,F
0,1,5444.476190,4890.714286,2.844594e+06,1.093484e+06,30087.330786,1.311362e+06,4.745881e+08,1.590539e+12,78.0,6.0,13.000000,5333.0,3641.0,1.464707,1212.0,786.0,837.0,84,0.015751,628.464286,25542.724915,6.220886,-4.503209,51.4,100.6,52,33,194.823017,696.854432,13.957902,26.398001,5.10,-1.70,0.337006,0.651409,-0.488091,0.367556,1.803719,6.891050,1.343026,2.625081,4,4,0
1,1,4680.800000,5325.541176,2.722303e+06,9.967454e+05,28010.484847,1.068252e+06,2.918423e+08,9.004063e+11,79.0,6.0,13.166667,5327.0,3434.0,1.551252,1192.0,732.0,802.0,85,0.015956,600.964706,16643.845813,6.858633,-7.156135,50.4,64.1,56,32,212.334730,456.021119,14.571710,21.354651,2.65,-5.45,0.459392,0.717547,-0.794716,0.528846,1.965452,4.286016,1.401946,2.070269,5,6,0
2,1,4845.088889,4865.122222,2.373370e+06,1.081331e+06,32769.607511,1.302167e+06,3.374732e+08,1.203559e+12,84.0,6.0,14.000000,5238.0,3700.0,1.415676,1262.0,780.0,809.0,90,0.017182,587.333333,22772.333333,7.909256,-6.923128,55.4,49.7,60,37,261.247515,524.372486,16.163153,22.899181,4.60,-3.80,0.489648,0.838460,-0.763180,0.562389,2.431361,5.003874,1.559282,2.236934,5,7,0
3,1,5785.963415,4738.292683,2.844040e+06,1.063888e+06,30366.535171,1.231116e+06,3.370106e+08,7.937888e+11,76.0,6.0,12.666667,5301.0,3577.0,1.481968,1202.0,812.0,837.0,82,0.015469,607.670732,25724.025729,7.353172,-7.807298,57.8,42.9,52,31,274.560531,492.668926,16.569868,22.196147,3.70,-3.90,0.470150,0.778553,-0.856554,0.540881,2.538565,4.640880,1.593287,2.154270,4,7,0
4,1,4799.870968,5569.516129,1.750609e+06,9.879794e+05,31919.071720,2.973347e+06,5.635749e+08,1.370647e+14,85.0,8.0,10.625000,4208.0,3615.0,1.164039,1352.0,831.0,832.0,93,0.022101,541.139785,26968.184761,5.607481,-5.236173,49.5,71.2,43,46,236.265098,579.975362,15.370917,24.082678,0.00,0.55,0.469780,0.499230,-1.464788,0.709110,3.611359,92.893264,1.900358,9.638115,6,5,0


In [9]:
df.shape

(1600, 45)

In [12]:
df.columns

Index(['ID', 'avgX', 'avgY', 'avgSDX', 'avgSDY', 'avgV', 'avgA', 'avgSDV',
       'avgSDA', 'pen_down', 'pen_up', 'pen_ratio', 'sign_width',
       'sign_height', 'width_height_ratio', 'total_sign_duration',
       'range_pressure', 'max_pressure', 'sample_points',
       'sample_points_to_width', 'mean_pressure', 'pressure_variance',
       'avg_x_velocity', 'avg_y_velocity', 'max_x_velocity', 'max_y_velocity',
       'samples_positive_x_velocity', 'samples_positive_y_velocity',
       'variance_x_velocity', 'variance_y_velocity', 'std_x_velocity',
       'std_y_velocity', 'median_x_velocity', 'median_y_velocity',
       'corr_x_y_velocity', 'mean_x_acceleration', 'mean_y_acceleration',
       'corr_x_y_acceleration', 'variance_x_acceleration',
       'variance_y_acceleration', 'std_x_acceleration', 'std_y_acceleration',
       'x_local_minima', 'y_local_minima', 'F'],
      dtype='object')

## Saving the feature vectors to the file

In [11]:
df.to_csv('../features.csv', index=False)